In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [9]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
niter = 500
rank = 80
tau = 3
gamma = 0.5
rho = 0.5
alpha = 1



print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
    d0 = 561 #561 =3*11*17

    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 512
    d6 = 6 


    limit = torch.sqrt(torch.tensor(3. / d0))
    W1 = 0.2* torch.empty(d1, d0, device=device).uniform_(-limit, limit)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    limit = torch.sqrt(torch.tensor(3. / d1))
    W2 = 0.2* torch.empty(d2, d1, device=device).uniform_(-limit, limit)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    limit = torch.sqrt(torch.tensor(3. / d2))
    W3 = 0.2* torch.empty(d3, d2, device=device).uniform_(-limit, limit)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    limit = torch.sqrt(torch.tensor(3. / d3))
    W4 = 0.2* torch.empty(d4, d3, device=device).uniform_(-limit, limit)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)

    limit = torch.sqrt(torch.tensor(3. / d4))
    W5 = 0.2* torch.empty(d5, d4, device=device).uniform_(-limit, limit)
    b5 = 0*torch.ones(d5, 1, device=device)
    W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())
    factors5 = tensor_train(W5_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W5_tl_tensor_rec = tt_to_tensor(factors5)
    b5 = 0*torch.ones(d5, 1, device=device)


    limit = torch.sqrt(torch.tensor(3. / d5))
    W6 = 0.2* torch.empty(d6, d5, device=device).uniform_(-limit, limit)
    b5 = 0*torch.ones(d5, 1, device=device)
    b6 = 0*torch.ones(d6, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = nn.ReLU()(U5)
    U6 = torch.addmm(b6.repeat(1, N), W6, V5)
    V6 = U6 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V6 = (y_one_hot + gamma*U6 + alpha*V6)/(1 + gamma + alpha)

        # update U4 
        U6 = (gamma*V6 + rho*(torch.mm(W6,V5) + b6.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W6, b6 = updateWb_org(U6,V5,W6,b6,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


        # update for 5th layer
        # update V3
        V5 = updateV(U5,U6,W6,b6,rho,gamma)

        # update U3
        U5 = relu_prox(V5,(rho*torch.addmm(b5.repeat(1,N), W5, V4) + alpha*U5)/(rho + alpha),(rho + alpha)/gamma,d5,N)

        # update W3 and b3
        W5, b5 = updateWb(U5,V4,W5,b5,W5_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4))
        W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors5 = tensor_train(W5_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W5_tl_tensor_rec = tt_to_tensor(factors5)


  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        a5_train = nn.ReLU()(torch.addmm(b5.repeat(1, N), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b6.repeat(1, N), W6, a5_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        a5_test = nn.ReLU()(torch.addmm(b5.repeat(1, N_test), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_test))
        pred_test = torch.argmax(torch.addmm(b6.repeat(1, N_test), W6, a5_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V6,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b6.repeat(1,N), W6, V5),U6,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,nn.ReLU()(U5),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V6,U6,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W5.reshape((4,4,4,4,4,4,4,4,4)),torch.as_tensor(W5_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))
    factors5_shape=[f.shape for f in factors5]
    Sum_of_variables_factors5=sum(list(x*y*z for x,y,z in factors5_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4+Sum_of_variables_factors5

    CR_1=((total_variabels)+(d5*d6))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5+d5*d6)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#     #this postion to add new row into existing table
#         df=pd.read_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
#         new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
#                    'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
#                    'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1],'seed':seed} 
#         df=df.append(new_row,ignore_index=True)
#         df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv',index=False)


filename= "LecunUniform_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/5 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 3 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 7.644079685211182 - sq_loss: 661.6734619140625 - tot_loss: 967.5735862401853 - acc: 0.17464635473340587 - val_acc: 0.166949440108585
Repeatition 1 Epoch 2 / 500 
 - time: 6.727517366409302 - sq_loss: 294.07708740234375 - tot_loss: 493.1037290778477 - acc: 0.19028835690968443 - val_acc: 0.17950458092975907
Repeatition 1 Epoch 3 / 500 
 - time: 6.609018087387085 - sq_loss: 164.3173065185547 - tot_loss: 269.73275418765843 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 4 / 500 
 - time: 6.653204679489136 - sq_loss: 89.59823608398438 - tot_loss: 152.41527014551684 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 5 / 500 
 - time: 6.505345582962036 - sq_loss: 48.27982711791992 - tot_loss: 89.25749735720456 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 6 / 500 
 - time: 6.71285557746

Repeatition 1 Epoch 48 / 500 
 - time: 5.129729986190796 - sq_loss: 0.00043711168109439313 - tot_loss: 1.5988660375296604 - acc: 0.580930359085963 - val_acc: 0.5385137427892772
Repeatition 1 Epoch 49 / 500 
 - time: 5.1277031898498535 - sq_loss: 0.00040948056266643107 - tot_loss: 1.5485789094345819 - acc: 0.5943960826985855 - val_acc: 0.5429250084832032
Repeatition 1 Epoch 50 / 500 
 - time: 5.041826963424683 - sq_loss: 0.0003840109275188297 - tot_loss: 1.5017026915247698 - acc: 0.6071817192600653 - val_acc: 0.5514082117407533
Repeatition 1 Epoch 51 / 500 
 - time: 4.992061614990234 - sq_loss: 0.000360411882866174 - tot_loss: 1.4399022844081628 - acc: 0.6179270946681176 - val_acc: 0.5612487275195114
Repeatition 1 Epoch 52 / 500 
 - time: 5.047180891036987 - sq_loss: 0.00033868217724375427 - tot_loss: 1.3982214765692333 - acc: 0.6297606093579978 - val_acc: 0.5714285714285714
Repeatition 1 Epoch 53 / 500 
 - time: 5.028551816940308 - sq_loss: 0.00031864718766883016 - tot_loss: 1.34030524

Repeatition 1 Epoch 95 / 500 
 - time: 4.982845067977905 - sq_loss: 3.354251748532988e-05 - tot_loss: 0.4283623342728333 - acc: 0.8204570184983678 - val_acc: 0.7838479809976246
Repeatition 1 Epoch 96 / 500 
 - time: 5.012707710266113 - sq_loss: 3.187010224792175e-05 - tot_loss: 0.4503126346232875 - acc: 0.8235854189336235 - val_acc: 0.7872412623006447
Repeatition 1 Epoch 97 / 500 
 - time: 5.030372619628906 - sq_loss: 3.0290555514511652e-05 - tot_loss: 0.4301670407214715 - acc: 0.8260337323177367 - val_acc: 0.7889379029521547
Repeatition 1 Epoch 98 / 500 
 - time: 4.905877351760864 - sq_loss: 2.881004911614582e-05 - tot_loss: 0.4310126974855848 - acc: 0.8280739934711643 - val_acc: 0.7896165592127588
Repeatition 1 Epoch 99 / 500 
 - time: 5.037894248962402 - sq_loss: 2.7424614017945714e-05 - tot_loss: 0.40450943405198814 - acc: 0.8310663764961915 - val_acc: 0.7923311842551748
Repeatition 1 Epoch 100 / 500 
 - time: 4.967034816741943 - sq_loss: 2.6093364795087837e-05 - tot_loss: 0.397894

Repeatition 1 Epoch 141 / 500 
 - time: 6.032577991485596 - sq_loss: 4.3825375541928224e-06 - tot_loss: 0.2456061164961909 - acc: 0.9068280739934712 - val_acc: 0.8737699355276553
Repeatition 1 Epoch 142 / 500 
 - time: 6.096586465835571 - sq_loss: 4.22602306571207e-06 - tot_loss: 0.22825243793209893 - acc: 0.9073721436343852 - val_acc: 0.8754665761791652
Repeatition 1 Epoch 143 / 500 
 - time: 6.081078290939331 - sq_loss: 4.076812274433905e-06 - tot_loss: 0.2406947679124869 - acc: 0.9081882480957563 - val_acc: 0.8768238887003733
Repeatition 1 Epoch 144 / 500 
 - time: 6.2123143672943115 - sq_loss: 3.9593032852280885e-06 - tot_loss: 0.23184564766961557 - acc: 0.9087323177366703 - val_acc: 0.8781812012215813
Repeatition 1 Epoch 145 / 500 
 - time: 6.190211772918701 - sq_loss: 3.8349903661583085e-06 - tot_loss: 0.24156170604901206 - acc: 0.9100924918389554 - val_acc: 0.8795385137427892
Repeatition 1 Epoch 146 / 500 
 - time: 6.216466665267944 - sq_loss: 3.7052479910926195e-06 - tot_loss: 

Repeatition 1 Epoch 187 / 500 
 - time: 6.265456438064575 - sq_loss: 1.5789031522217556e-06 - tot_loss: 0.18594287649230168 - acc: 0.9315832426550599 - val_acc: 0.9104173736002714
Repeatition 1 Epoch 188 / 500 
 - time: 6.226737022399902 - sq_loss: 1.555799258312618e-06 - tot_loss: 0.20571352578768654 - acc: 0.9315832426550599 - val_acc: 0.9104173736002714
Repeatition 1 Epoch 189 / 500 
 - time: 6.261124849319458 - sq_loss: 1.5378976740976213e-06 - tot_loss: 0.1834360699415747 - acc: 0.9319912948857454 - val_acc: 0.9104173736002714
Repeatition 1 Epoch 190 / 500 
 - time: 6.100969076156616 - sq_loss: 1.5257131735779694e-06 - tot_loss: 0.18447294953096005 - acc: 0.9325353645266594 - val_acc: 0.9104173736002714
Repeatition 1 Epoch 191 / 500 
 - time: 6.304095268249512 - sq_loss: 1.507446881987562e-06 - tot_loss: 0.18140315423261733 - acc: 0.9323993471164309 - val_acc: 0.9107567017305734
Repeatition 1 Epoch 192 / 500 
 - time: 6.252147912979126 - sq_loss: 1.4957605571908061e-06 - tot_loss:

...

Repeatition 1 Epoch 279 / 500 
 - time: 3.2423088550567627 - sq_loss: 9.608518212189665e-07 - tot_loss: 0.16628539017309096 - acc: 0.9628672470076169 - val_acc: 0.9402782490668476
Repeatition 1 Epoch 280 / 500 
 - time: 3.2040963172912598 - sq_loss: 9.582397524354747e-07 - tot_loss: 0.16945469109342204 - acc: 0.963411316648531 - val_acc: 0.9392602646759416
Repeatition 1 Epoch 281 / 500 
 - time: 3.216423273086548 - sq_loss: 9.544411341266823e-07 - tot_loss: 0.16455096230503852 - acc: 0.9632752992383025 - val_acc: 0.9395995928062436
Repeatition 1 Epoch 282 / 500 
 - time: 3.164005756378174 - sq_loss: 9.502323905508092e-07 - tot_loss: 0.17592927863622743 - acc: 0.963411316648531 - val_acc: 0.9392602646759416
Repeatition 1 Epoch 283 / 500 
 - time: 3.260082960128784 - sq_loss: 9.472611282035359e-07 - tot_loss: 0.1780817493348188 - acc: 0.9635473340587595 - val_acc: 0.9395995928062436
Repeatition 1 Epoch 284 / 500 
 - time: 3.1269092559814453 - sq_loss: 9.442240980206407e-07 - tot_loss: 0.

Repeatition 1 Epoch 325 / 500 
 - time: 3.117217779159546 - sq_loss: 8.501664865434577e-07 - tot_loss: 0.1564373897600433 - acc: 0.9715723612622416 - val_acc: 0.9457074991516796
Repeatition 1 Epoch 326 / 500 
 - time: 3.1792988777160645 - sq_loss: 8.485967555316165e-07 - tot_loss: 0.15867999860029647 - acc: 0.9715723612622416 - val_acc: 0.9463861554122837
Repeatition 1 Epoch 327 / 500 
 - time: 3.1569385528564453 - sq_loss: 8.447638606412511e-07 - tot_loss: 0.17082467853125882 - acc: 0.9718443960826986 - val_acc: 0.9463861554122837
Repeatition 1 Epoch 328 / 500 
 - time: 3.306798219680786 - sq_loss: 8.433882499048195e-07 - tot_loss: 0.17277056093184484 - acc: 0.9719804134929271 - val_acc: 0.9467254835425857
Repeatition 1 Epoch 329 / 500 
 - time: 3.2134835720062256 - sq_loss: 8.423354529440985e-07 - tot_loss: 0.16466232254087654 - acc: 0.9719804134929271 - val_acc: 0.9460468272819816
Repeatition 1 Epoch 330 / 500 
 - time: 3.2141895294189453 - sq_loss: 8.40738493934623e-07 - tot_loss: 

Repeatition 1 Epoch 371 / 500 
 - time: 3.2279200553894043 - sq_loss: 7.762483846818213e-07 - tot_loss: 0.16437227180869285 - acc: 0.9759249183895539 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 372 / 500 
 - time: 3.2224624156951904 - sq_loss: 7.745174457340909e-07 - tot_loss: 0.18267774586230945 - acc: 0.9760609357997824 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 373 / 500 
 - time: 3.01023530960083 - sq_loss: 7.719679047113459e-07 - tot_loss: 0.16358278429962603 - acc: 0.9760609357997824 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 374 / 500 
 - time: 1.5449066162109375 - sq_loss: 7.691417067690054e-07 - tot_loss: 0.18231415784642113 - acc: 0.9760609357997824 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 375 / 500 
 - time: 1.5467078685760498 - sq_loss: 7.663351766495907e-07 - tot_loss: 0.1501851885969303 - acc: 0.9764689880304679 - val_acc: 0.9514760773668137
Repeatition 1 Epoch 376 / 500 
 - time: 1.5598742961883545 - sq_loss: 7.646114568160556e-07 - tot_loss:

Repeatition 1 Epoch 417 / 500 
 - time: 1.5436627864837646 - sq_loss: 7.109979378583375e-07 - tot_loss: 0.16535616168847045 - acc: 0.9795973884657236 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 418 / 500 
 - time: 1.5520658493041992 - sq_loss: 7.091869065334322e-07 - tot_loss: 0.1758541185534026 - acc: 0.9797334058759521 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 419 / 500 
 - time: 1.5567665100097656 - sq_loss: 7.085906190695823e-07 - tot_loss: 0.17569619305857853 - acc: 0.9798694232861807 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 420 / 500 
 - time: 1.5527708530426025 - sq_loss: 7.070043466228526e-07 - tot_loss: 0.15799502095114182 - acc: 0.9798694232861807 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 421 / 500 
 - time: 1.532303810119629 - sq_loss: 7.061863698254456e-07 - tot_loss: 0.16684545455362643 - acc: 0.9798694232861807 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 422 / 500 
 - time: 1.5200815200805664 - sq_loss: 7.047611347843485e-07 - tot_loss

Repeatition 1 Epoch 463 / 500 
 - time: 1.5218446254730225 - sq_loss: 6.618114412049181e-07 - tot_loss: 0.17833529414497584 - acc: 0.9823177366702938 - val_acc: 0.9541907024092298
Repeatition 1 Epoch 464 / 500 
 - time: 1.5465373992919922 - sq_loss: 6.609977845073445e-07 - tot_loss: 0.15509233221579533 - acc: 0.9823177366702938 - val_acc: 0.9541907024092298
Repeatition 1 Epoch 465 / 500 
 - time: 1.5188405513763428 - sq_loss: 6.609541856050782e-07 - tot_loss: 0.16959787801775517 - acc: 0.9821817192600653 - val_acc: 0.9538513742789277
Repeatition 1 Epoch 466 / 500 
 - time: 1.5645525455474854 - sq_loss: 6.59040608752548e-07 - tot_loss: 0.16899065098689192 - acc: 0.9824537540805223 - val_acc: 0.9541907024092298
Repeatition 1 Epoch 467 / 500 
 - time: 1.5319688320159912 - sq_loss: 6.575300517397409e-07 - tot_loss: 0.17205564606094637 - acc: 0.9823177366702938 - val_acc: 0.9535120461486257
Repeatition 1 Epoch 468 / 500 
 - time: 1.5541071891784668 - sq_loss: 6.565289254467643e-07 - tot_los

In [ ]:
seed